# Language Model

In [39]:
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

In [40]:
import nltk
nltk.download('reuters')
nltk.download('punkt')

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\vivek\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vivek\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [41]:
# Create a placeholder for model
model = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurance  
for sentence in reuters.sents():
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1
 
# Let's transform the counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

In [42]:
model

defaultdict(<function __main__.<lambda>()>,
            {(None,
              None): defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'ASIAN': 7.31047591198187e-05,
                          'They': 0.008151180641859785,
                          'But': 0.019263104028072228,
                          'The': 0.16154324146501936,
                          'Unofficial': 1.8276189779954676e-05,
                          '"': 0.06559324512025733,
                          'In': 0.02522114189633745,
                          'Threat': 3.655237955990935e-05,
                          'Taiwan': 0.0006944952116382777,
                          'Retaliation': 5.482856933986402e-05,
                          'A': 0.013963008991885371,
                          'Last': 0.0036917903355508444,
                          'Much': 0.0001462095182396374,
                          'He': 0.028986036991008116,
                          'Meanwhile': 0.000749323780978141

In [43]:
dict(model['the', 'price'])

{'yesterday': 0.004651162790697674,
 'of': 0.3209302325581395,
 'it': 0.05581395348837209,
 'effect': 0.004651162790697674,
 'cut': 0.009302325581395349,
 'for': 0.05116279069767442,
 'paid': 0.013953488372093023,
 'to': 0.05581395348837209,
 'increases': 0.013953488372093023,
 'used': 0.004651162790697674,
 'climate': 0.004651162790697674,
 '.': 0.023255813953488372,
 'cuts': 0.009302325581395349,
 'reductions': 0.004651162790697674,
 'limit': 0.004651162790697674,
 'now': 0.004651162790697674,
 'moved': 0.004651162790697674,
 'per': 0.013953488372093023,
 'adjustments': 0.004651162790697674,
 '(': 0.009302325581395349,
 'slumped': 0.004651162790697674,
 'is': 0.018604651162790697,
 'move': 0.004651162790697674,
 'evolution': 0.004651162790697674,
 'differentials': 0.009302325581395349,
 'went': 0.004651162790697674,
 'the': 0.013953488372093023,
 'factor': 0.004651162790697674,
 'Royal': 0.004651162790697674,
 ',': 0.018604651162790697,
 'again': 0.004651162790697674,
 'changes': 0.0

In [44]:
sorted(dict(model["the","news"]).items(), key=lambda x:-1*x[1])

[('conference', 0.25),
 ('of', 0.125),
 ('.', 0.125),
 ('with', 0.08333333333333333),
 (',', 0.08333333333333333),
 ('agency', 0.08333333333333333),
 ('that', 0.08333333333333333),
 ('brought', 0.041666666666666664),
 ('about', 0.041666666666666664),
 ('broke', 0.041666666666666664),
 ('on', 0.041666666666666664)]

In [56]:
import random

# text = ['today','the']
# text = ['the', 'price']
text = ["the", "news"]
sentence_finished = False

while not sentence_finished:
    r = random.random()
#     print(r)
#     print('Accumulator: ', accumulator)
    accumulator = .0
    
    for word in model[tuple(text[-2:])].keys():
        accumulator += model[tuple(text[-2:])][word]
        
        if accumulator >=r:
            text.append(word)
            break
    if text[-2:] == [None, None]:
        sentence_finished = True

print(' '.join([t for t in text if t]))

the news with West Germany .


# Neural Language Model

In [69]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical, plot_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
import numpy as np

# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # pre-pad sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
        # predict probabilities for each word
        yhat = model.predict(encoded, verbose=0)
        yhat = np.argmax(yhat,axis=1)
        
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
    return in_text

# source text
data = """ Jack and Jill went up the hill\n
            To fetch a pail of water\n
            Jack fell down and broke his crown\n
            And Jill came tumbling after\n """

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]
# retrieve vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# encode 2 words -> 1 word
sequences = list()
for i in range(2, len(encoded)):
    sequence = encoded[i-2:i+1]
    sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)
# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=500, verbose=2)

Vocabulary Size: 22
Total Sequences: 23
Max Sequence Length: 3
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 2, 10)             220       
                                                                 
 lstm_3 (LSTM)               (None, 50)                12200     
                                                                 
 dense_3 (Dense)             (None, 22)                1122      
                                                                 
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
1/1 - 2s - loss: 3.0923 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
Epoch 2/500
1/1 - 0s - loss: 3.0914 - accuracy: 0.0435 - 5ms/epoch - 5ms/step
Epoch 3/500
1/1 - 0s - loss: 3.0905 - accuracy: 0.1304 - 6ms/epoch - 6ms/step
Epoch 4/50

Epoch 94/500
1/1 - 0s - loss: 2.5370 - accuracy: 0.1304 - 6ms/epoch - 6ms/step
Epoch 95/500
1/1 - 0s - loss: 2.5176 - accuracy: 0.1739 - 4ms/epoch - 4ms/step
Epoch 96/500
1/1 - 0s - loss: 2.4980 - accuracy: 0.1739 - 4ms/epoch - 4ms/step
Epoch 97/500
1/1 - 0s - loss: 2.4780 - accuracy: 0.1739 - 5ms/epoch - 5ms/step
Epoch 98/500
1/1 - 0s - loss: 2.4577 - accuracy: 0.1739 - 5ms/epoch - 5ms/step
Epoch 99/500
1/1 - 0s - loss: 2.4372 - accuracy: 0.1739 - 4ms/epoch - 4ms/step
Epoch 100/500
1/1 - 0s - loss: 2.4165 - accuracy: 0.1739 - 4ms/epoch - 4ms/step
Epoch 101/500
1/1 - 0s - loss: 2.3955 - accuracy: 0.1739 - 6ms/epoch - 6ms/step
Epoch 102/500
1/1 - 0s - loss: 2.3742 - accuracy: 0.2174 - 4ms/epoch - 4ms/step
Epoch 103/500
1/1 - 0s - loss: 2.3528 - accuracy: 0.2174 - 3ms/epoch - 3ms/step
Epoch 104/500
1/1 - 0s - loss: 2.3313 - accuracy: 0.2174 - 6ms/epoch - 6ms/step
Epoch 105/500
1/1 - 0s - loss: 2.3095 - accuracy: 0.2174 - 4ms/epoch - 4ms/step
Epoch 106/500
1/1 - 0s - loss: 2.2876 - accura

Epoch 197/500
1/1 - 0s - loss: 0.5013 - accuracy: 0.9565 - 9ms/epoch - 9ms/step
Epoch 198/500
1/1 - 0s - loss: 0.4900 - accuracy: 0.9565 - 6ms/epoch - 6ms/step
Epoch 199/500
1/1 - 0s - loss: 0.4788 - accuracy: 0.9565 - 7ms/epoch - 7ms/step
Epoch 200/500
1/1 - 0s - loss: 0.4680 - accuracy: 0.9565 - 5ms/epoch - 5ms/step
Epoch 201/500
1/1 - 0s - loss: 0.4575 - accuracy: 0.9565 - 9ms/epoch - 9ms/step
Epoch 202/500
1/1 - 0s - loss: 0.4472 - accuracy: 0.9565 - 9ms/epoch - 9ms/step
Epoch 203/500
1/1 - 0s - loss: 0.4372 - accuracy: 0.9565 - 7ms/epoch - 7ms/step
Epoch 204/500
1/1 - 0s - loss: 0.4274 - accuracy: 0.9565 - 8ms/epoch - 8ms/step
Epoch 205/500
1/1 - 0s - loss: 0.4179 - accuracy: 0.9565 - 6ms/epoch - 6ms/step
Epoch 206/500
1/1 - 0s - loss: 0.4086 - accuracy: 0.9565 - 6ms/epoch - 6ms/step
Epoch 207/500
1/1 - 0s - loss: 0.3996 - accuracy: 0.9565 - 7ms/epoch - 7ms/step
Epoch 208/500
1/1 - 0s - loss: 0.3909 - accuracy: 0.9565 - 6ms/epoch - 6ms/step
Epoch 209/500
1/1 - 0s - loss: 0.3823 - 

Epoch 300/500
1/1 - 0s - loss: 0.1155 - accuracy: 0.9565 - 3ms/epoch - 3ms/step
Epoch 301/500
1/1 - 0s - loss: 0.1149 - accuracy: 0.9565 - 6ms/epoch - 6ms/step
Epoch 302/500
1/1 - 0s - loss: 0.1142 - accuracy: 0.9565 - 5ms/epoch - 5ms/step
Epoch 303/500
1/1 - 0s - loss: 0.1135 - accuracy: 0.9565 - 7ms/epoch - 7ms/step
Epoch 304/500
1/1 - 0s - loss: 0.1129 - accuracy: 0.9565 - 5ms/epoch - 5ms/step
Epoch 305/500
1/1 - 0s - loss: 0.1123 - accuracy: 0.9565 - 6ms/epoch - 6ms/step
Epoch 306/500
1/1 - 0s - loss: 0.1116 - accuracy: 0.9565 - 5ms/epoch - 5ms/step
Epoch 307/500
1/1 - 0s - loss: 0.1110 - accuracy: 0.9565 - 5ms/epoch - 5ms/step
Epoch 308/500
1/1 - 0s - loss: 0.1104 - accuracy: 0.9565 - 5ms/epoch - 5ms/step
Epoch 309/500
1/1 - 0s - loss: 0.1098 - accuracy: 0.9565 - 4ms/epoch - 4ms/step
Epoch 310/500
1/1 - 0s - loss: 0.1093 - accuracy: 0.9565 - 5ms/epoch - 5ms/step
Epoch 311/500
1/1 - 0s - loss: 0.1087 - accuracy: 0.9565 - 4ms/epoch - 4ms/step
Epoch 312/500
1/1 - 0s - loss: 0.1082 - 

Epoch 403/500
1/1 - 0s - loss: 0.0819 - accuracy: 0.9565 - 5ms/epoch - 5ms/step
Epoch 404/500
1/1 - 0s - loss: 0.0818 - accuracy: 0.9565 - 4ms/epoch - 4ms/step
Epoch 405/500
1/1 - 0s - loss: 0.0816 - accuracy: 0.9565 - 6ms/epoch - 6ms/step
Epoch 406/500
1/1 - 0s - loss: 0.0815 - accuracy: 0.9565 - 5ms/epoch - 5ms/step
Epoch 407/500
1/1 - 0s - loss: 0.0814 - accuracy: 0.9565 - 4ms/epoch - 4ms/step
Epoch 408/500
1/1 - 0s - loss: 0.0812 - accuracy: 0.9565 - 4ms/epoch - 4ms/step
Epoch 409/500
1/1 - 0s - loss: 0.0811 - accuracy: 0.9565 - 10ms/epoch - 10ms/step
Epoch 410/500
1/1 - 0s - loss: 0.0809 - accuracy: 0.9565 - 4ms/epoch - 4ms/step
Epoch 411/500
1/1 - 0s - loss: 0.0808 - accuracy: 0.9565 - 7ms/epoch - 7ms/step
Epoch 412/500
1/1 - 0s - loss: 0.0807 - accuracy: 0.9565 - 7ms/epoch - 7ms/step
Epoch 413/500
1/1 - 0s - loss: 0.0805 - accuracy: 0.9565 - 5ms/epoch - 5ms/step
Epoch 414/500
1/1 - 0s - loss: 0.0804 - accuracy: 0.9565 - 5ms/epoch - 5ms/step
Epoch 415/500
1/1 - 0s - loss: 0.0803 

In [74]:
# evaluate model
print(generate_seq(model, tokenizer, max_length-1, 'Jack and', 15))
print(generate_seq(model, tokenizer, max_length-1, 'And Jill', 3))
print(generate_seq(model, tokenizer, max_length-1, 'fell down', 5))
print(generate_seq(model, tokenizer, max_length-1, 'pail of', 5))

Jack and jill went up the hill to fetch a pail of water jack fell down and
And Jill went up the
fell down and broke his crown and
pail of water jack fell down and
